In [8]:
import alpaca_trade_api as tradeapi
import pandas as pd
from tabulate import tabulate


import os
from dotenv import load_dotenv, find_dotenv
from pathlib import Path

In [9]:
def get_credentials():
    load_dotenv("creds.env")
    api_key = os.getenv("ALPACA_API_KEY")
    secret_key = os.getenv("SECRET_KEY")
    if not api_key and not secret_key:
        print("API key not found. Please check your creds.env file.")
    else:
        api = tradeapi.REST(api_key, secret_key, base_url='https://paper-api.alpaca.markets')
        print("API key loaded successfully.")
    return api

## Placing Orders

In [10]:
api = get_credentials()

def place_order(symbol, qty, order_type="market", side="buy"):
    """
    Places an order for the specified stock symbol.

    Args:
        symbol (str): Stock symbol (e.g., 'AAPL', 'TSLA').
        qty (int): Number of shares to buy/sell.
        order_type (str, optional): Order type ('market' by default).
        side (str, optional): 'buy' or 'sell' (default is 'buy').

    Returns:
        None
    """
    try:
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side=side,
            type=order_type,
            time_in_force='gtc'
        )
        print(f"Order placed: {side} {qty} shares of {symbol}")
    except Exception as e:
        print(f"Error placing order: {e}")




API key loaded successfully.
